In [14]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from collections import Counter
import itertools

pd.set_option('display.max_colwidth', 180)
pd.set_option('expand_frame_repr', False)

In [71]:
csvfile = '../proc_data_phase0/verification/all_responses_round0-3_verified.csv'
df = pd.read_csv(csvfile, 
    sep="\t")

total_objects=len(df)
print("Total objects: ", total_objects)
df.columns

Total objects:  25315


Index(['vg_img_id', 'cat', 'synset', 'vg_obj_name', 'responses_r0', 'opt-outs',
       'vg_domain', 'top_response_domain_r0', 'responses_domains_r0', 'url',
       'sample_type', 'responses_domains_r1', 'responses_r1',
       'top_response_domain_r1', 'responses_domains_r2', 'responses_r2',
       'top_response_domain_r2', 'responses_domains_r3', 'responses_r3',
       'top_response_domain_r3', 'vg_object_id', 'all_responses', 'clean',
       'canon', 'spellchecked', 'spellchecked_min2', 'verified'],
      dtype='object')

In [72]:
df['clean'] = df['clean'].apply(lambda x: eval(x))
df['verified'] = df['verified'].apply(lambda x: eval(x))
df['canon'] = df['canon'].apply(lambda x: eval(x))


In [73]:
df['canon'][0]

Counter({'man': 11,
         'batter': 8,
         'player': 5,
         'person': 4,
         'baseball player': 2,
         'helmet': 2,
         'boy': 1,
         'uniform': 1,
         'bat': 1})

In [74]:
set(df['cat'])

{'animal',
 'article of clothing',
 'food, nutrient',
 'food, solid food',
 'instrumentality, instrumentation',
 'person',
 'plant, flora, plant life',
 'structure, construction',
 'tableware',
 'tool',
 'vehicle',
 'ware'}

In [75]:
df['mn_name'] = df['canon'].apply(lambda x: x.most_common(1)[0][0])

In [76]:
vocab = set(df.mn_name)

In [77]:
vocab

{'airplane',
 'airport',
 'ambulance',
 'animal',
 'apple',
 'apron',
 'aquarium',
 'arm',
 'athlete',
 'awning',
 'baby',
 'backpack',
 'bacon',
 'bagel',
 'balloon',
 'banana',
 'bar',
 'barn',
 'baseball player',
 'basketball player',
 'bat',
 'bathtub',
 'batter',
 'beach',
 'bear',
 'beard',
 'bed',
 'beer',
 'bench',
 'benches',
 'bidet',
 'bike',
 'bird',
 'bird bath',
 'birdbath',
 'biscuit',
 'blanket',
 'blender',
 'board',
 'boat',
 'book',
 'bookshelf',
 'booth',
 'bottle',
 'bowl',
 'bowtie',
 'box',
 'boy',
 'bread',
 'bricks',
 'bride',
 'bridge',
 'broccoli',
 'bucket',
 'buffet',
 'bug',
 'building',
 'bull',
 'bun',
 'bunk bed',
 'bunkbed',
 'burger',
 'burrito',
 'bus',
 'bus stop',
 'butterfly',
 'cabbage',
 'cabinet',
 'cabinets',
 'cake',
 'camera',
 'can',
 'canister',
 'canopy',
 'cape',
 'car',
 'carpet',
 'carriage',
 'carrot',
 'cart',
 'castle',
 'cat',
 'catcher',
 'cereal',
 'chair',
 'cheese',
 'cheetah',
 'chicken',
 'child',
 'chili',
 'china',
 'church

In [78]:
len(vocab)

445

In [79]:

error_cats = Counter()

for ix,row in df.iterrows():
    resp = row['canon']
    topname = resp.most_common(1)[0][0]
    #print(topname)
    for name in resp:
        if name == topname:
            error_cats['hit'] += resp[name]
        elif name not in row['verified']:
            error_cats['low_count'] += resp[name]
        else:
            if topname in row['verified'][name]['can_be_same_object']:
                error_cats['adequate_same_object'] += resp[name]
            else:
                if not row['verified'][name]['inadequacy_type']:
                    error_cats['adequate_diff_object'] += resp[name]
                else:
                     error_cats[row['verified'][name]['inadequacy_type']] += resp[name]

                    
                    
                    #print('img:',row['vg_img_id'],'--entry-level:',topname)
                    #print('less-preferred:',name)
                    #print(row['verified'][name])
            
    #if ix > 5:       
    #    break
        

In [80]:
error_cats

Counter({'hit': 616370,
         'adequate_same_object': 143690,
         'bounding box': 42971,
         'low_count': 80846,
         'other': 696,
         'adequate_diff_object': 7949,
         'visual': 1040,
         'linguistic': 106})

In [12]:
for t in error_cats:
    print(t,error_cats[t]/sum(error_cats.values()))

hit 0.6897080347511604
adequate_same_object 0.160786779877986
bounding box 0.04808385216881437
low_count 0.09046536297595975
other 0.0007788127134461568
adequate_diff_object 0.008894802096528017
visual 0.0011637431350344871
linguistic 0.00011861228107082273


In [81]:
csvfile = '../papers/ACL2020/material_experiments/FRCNN--VG1600--VGMN_MN.tsv'
csvfile2 = '../papers/ACL2020/material_experiments/FRCNN--VG1600_VG.tsv'

dfsys = pd.read_csv(csvfile, 
    sep="\t")

total_objects=len(dfsys)
print("Total objects: ", total_objects)
dfsys.columns

dfsys2 = pd.read_csv(csvfile2, 
    sep="\t")

total_objects=len(dfsys2)
print("Total objects: ", total_objects)
dfsys2.columns

Total objects:  1071
Total objects:  1071


Index(['image_id', 'category', 'mn_name', 'adequacy', 'pred_name',
       'adequacy.1', 'pred_score_converted'],
      dtype='object')

In [82]:
dfsys

,image_id,category,mn_name,adequacy,pred_name,adequacy.1,pred_score_converted
0,32,hit,cabinet,1.000000,cabinet,1.0,0.005396
1,160,hit,dress,1.000000,dress,1.0,0.005525
2,308,hit,bridge,1.000000,bridge,1.0,0.005896
3,474,error,table,1.000000,bed,--,0.005595
4,535,hit,table,1.000000,table,1.0,0.007436
5,556,hit,rug,1.000000,rug,1.0,0.007391
6,625,error,car,1.000000,van,--,0.003202
7,691,hyponym,animal,1.000000,giraffe,--,0.004862
8,1045,hit,table,1.000000,table,1.0,0.009762
9,1074,hit,house,1.000000,house,1.0,0.006800


In [15]:
df.query('vg_img_id == 32')

,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,url,...,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,spellchecked_min2,verified,mn_name
21087,32,"instrumentality, instrumentation",furnishing.n.02,cabinet,"{'cabinet': 2, 'wood': 1, 'sink': 1, 'cabinets': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}",{'#bbox': 1},home,home,"{'home': 0.25, '$wood': 0.125, '$sink': 0.125, '$cabinets': 0.125, '$cabinetry': 0.125, '$modern kitchen': 0.125, '$wall': 0.125}",http://object-naming-amore.upf.edu//32_1058643_seed_ambiguous.png,...,"{'cabinet': 4, 'cabinets': 2, 'cupboards': 2, 'cupboard': 1}",home,1058643,"Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1})","{'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","{'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1})","Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2})","{'cabinets': {'cluster': ('cabinet', 'cabinets', 'cupboard'), 'can_be_same_object': ('cabinet', 'cabinets', 'cupboard'), 'adequacy': 1.0, 'inadequacy_type': None, 'cluster_id':...",cabinet


In [90]:
def get_error(topname,predname,verified,response):
    if predname == topname:
        return 'hit'
    if predname not in verified:
        if predname in response:
            return 'low_count'
        else:
            return 'off???'
    if topname in verified[predname]['can_be_same_object']:
        return 'adequate'
    if not verified[predname]['inadequacy_type']:
        return 'adequate_diff_object'
    
    return verified[predname]['inadequacy_type']

In [91]:
hum_error_cats = Counter()
sys_error_cats = Counter()
sys_error_cats2 = Counter()


for ix,row in dfsys.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = resp.most_common(1)[0][0]
    verif = rowh['verified']
    
    if rowh['cat'] != 'person':

    #print(topname)
        for name in resp:
            if name in vocab:
                err = get_error(topname,name,verif,resp)
                hum_error_cats[err] += resp[name]
        
        sys_err = get_error(topname,row['pred_name'],verif,resp)
        sys_error_cats[sys_err] += 1
    
    
for ix,row in dfsys2.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = resp.most_common(1)[0][0]
    verif = rowh['verified']
    if rowh['cat'] != 'person':
        
        sys_err = get_error(topname,row['pred_name'],verif,resp)
        sys_error_cats2[sys_err] += 1

In [61]:
len(df)

25315

In [62]:
df

,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,url,...,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,spellchecked_min2,verified,mn_name
0,2357077,person,athlete.n.01,batter,"{'man': 4, 'batter': 2, 'baseball player': 1, 'player': 1, 'person': 1}",{},people,people,"{'people': 0.889, '$baseball player': 0.111}",http://object-naming-amore.upf.edu//2357077_815223_singleton_obj.png,...,"{'player': 3, 'man': 2, 'batter': 1, 'bat': 1, 'helmet': 1}",people,815223,"Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","{'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1}","{'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1}","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2})","{'batter': {'cluster': ('baseball player', 'batter', 'man', 'person', 'player'), 'can_be_same_object': ('baseball player', 'batter', 'man', 'person', 'player'), 'adequacy': 1.0...",man
1,2378759,article of clothing,overgarment.n.01,jacket,"{'jacket': 3, 'sweatshirt': 2, 'man': 2, 'hoodie': 1, 'person': 1}",{},clothing,clothing,"{'clothing': 0.333, '$sweatshirt': 0.222, 'people': 0.333, '$hoodie': 0.111}",http://object-naming-amore.upf.edu//2378759_1365136_supercat_unique.png,...,"{'jacket': 4, 'man': 2, 'sweatshirt': 1, 'hoodie': 1, 'shirt': 1}",clothing,1365136,"Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","{'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1}","{'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1}","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2})","{'hoodie': {'cluster': ('hoodie', 'jacket', 'sweatshirt'), 'can_be_same_object': ('hoodie', 'jacket', 'sweatshirt'), 'adequacy': 1.0, 'inadequacy_type': None, 'cluster_id': 0, ...",jacket
2,2319953,animal,ungulate.n.01,giraffe,{'giraffe': 7},{'#bbox': 2},animals_plants,animals_plants,{'animals_plants': 1.0},http://object-naming-amore.upf.edu//2319953_3089834_singleton_obj.png,...,"{'giraffe': 8, 'woman': 1}",animals_plants,3089834,"Counter({'giraffe': 32, 'woman': 1})","{'giraffe': 32, 'woman': 1}","{'giraffe': 32, 'woman': 1}","Counter({'giraffe': 32, 'woman': 1})",Counter({'giraffe': 32}),"{'giraffe': {'cluster': ['giraffe'], 'adequacy': 1.0, 'can_be_same_object': ['giraffe'], 'inadequacy_type': None, 'cluster_id': 0, 'cluster_weight': 1.0}}",giraffe
3,2361401,animal,ungulate.n.01,giraffe,{'giraffe': 9},{},animals_plants,animals_plants,{'animals_plants': 1.0},http://object-naming-amore.upf.edu//2361401_2363103_singleton_obj.png,...,{'giraffe': 9},animals_plants,2363103,Counter({'giraffe': 36}),{'giraffe': 36},{'giraffe': 36},Counter({'giraffe': 36}),Counter({'giraffe': 36}),"{'giraffe': {'cluster': ['giraffe'], 'adequacy': 1.0, 'can_be_same_object': ['giraffe'], 'inadequacy_type': None, 'cluster_id': 0, 'cluster_weight': 1.0}}",giraffe
4,2401994,"food, nutrient",dish.n.02,sandwich,"{'sandwich': 6, 'food': 2, 'hotdog': 1}",{},food,food,"{'food': 0.889, '$hotdog': 0.111}",http://object-naming-amore.upf.edu//2401994_1141828_singleton_obj.png,...,"{'food': 3, 'sandwich': 3, 'foot': 1, 'cake': 1, 'burger': 1}",food,1141828,"Counter({'sandwich': 25, 'food': 6, 'hotdog': 1, 'sandwhich': 1, 'foot': 1, 'cake': 1, 'burger': 1})","{'sandwich': 25, 'food': 6, 'hotdog': 1, 'sandwhich': 1, 'foot': 1, 'cake': 1, 'burger': 1}","{'sandwich': 26, 'food': 7, 'hotdog': 1, 'cake': 1, 'burger': 1}","Counter({'sandwich':

In [52]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
    ["humans"]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["FRCNN, no finetuning"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["FRCNN, finetuning on MN"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]

In [53]:
pd.DataFrame(table,columns=['(442-vocab)']+relevant_cats)

,(442-vocab),hit,adequate,low_count,bounding box,off???
0,humans,0.796357,0.116576,0.036565,0.041500,0.000000
1,"FRCNN, no finetuning",0.760992,0.109357,0.021421,0.022548,0.077790
2,"FRCNN, finetuning on MN",0.808343,0.076663,0.015784,0.010147,0.083427


In [56]:
sys_error_cats

Counter({'hit': 717,
         'off???': 74,
         'adequate': 68,
         'low_count': 14,
         'adequate_diff_object': 4,
         'bounding box': 9,
         'visual': 1})

In [57]:
sys_error_cats2

Counter({'adequate': 97,
         'off???': 69,
         'hit': 675,
         'bounding box': 20,
         'adequate_diff_object': 5,
         'low_count': 19,
         'visual': 2})

In [58]:
sum(sys_error_cats.values())

887

In [59]:
sum(hum_error_cats.values())

28771

In [60]:
hum_error_cats

Counter({'hit': 22912,
         'adequate': 3354,
         'low_count': 1052,
         'bounding box': 1194,
         'adequate_diff_object': 190,
         'visual': 58,
         'other': 11})

In [31]:
hum2_error_cats = Counter()
sys3_error_cats = Counter()
sys4_error_cats = Counter()


for ix,row in dfsys.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = rowh['vg_obj_name']
    verif = rowh['verified']
    
    if rowh['cat'] == 'person':

    #print(topname)
        for name in resp:
            if name in vocab:
                err = get_error(topname,name,verif,resp)
                hum2_error_cats[err] += resp[name]
        
        sys_err = get_error(topname,row['pred_name'],verif,resp)
        sys3_error_cats[sys_err] += 1
    
for ix,row in dfsys2.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
   
    topname = rowh['vg_obj_name']
    verif = rowh['verified']
    
    if rowh['cat'] == 'person':
        sys_err = get_error(topname,row['pred_name'],verif,resp)
        sys4_error_cats[sys_err] += 1

In [35]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
   # ["VG","human",""]+[hum2_error_cats[t]/sum(hum2_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "-"]+[sys4_error_cats[t]/sum(sys4_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "+"]+[sys3_error_cats[t]/sum(sys3_error_cats.values()) for t in relevant_cats],
    ["MN","human",""]+[hum2_error_cats[t]/sum(hum2_error_cats.values()) for t in relevant_cats],
    ["MN","FRCNN", "-"]+[sys3_error_cats[t]/sum(sys3_error_cats.values()) for t in relevant_cats],
    ["MN","FRCNN", "+"]+[sys4_error_cats[t]/sum(sys4_error_cats.values()) for t in relevant_cats]
]
rdf = pd.DataFrame(table,columns=['Test on','model', 'finetuning']+relevant_cats)

In [36]:
rdf.round(3)

,Test on,model,finetuning,hit,adequate,low_count,bounding box,off???
0,MN,human,,0.432,0.434,0.070,0.055,0.000
1,MN,FRCNN,-,0.522,0.375,0.027,0.011,0.060
2,MN,FRCNN,+,0.527,0.353,0.033,0.000,0.076


In [15]:
smatx = np.loadtxt("scores.csv",delimiter=',')

In [16]:
smatx.shape

(25310, 1601)

In [48]:
avocab = [f.strip() for f in open("aliased-objects_vocab.txt","r").readlines()]

In [49]:
len(avocab)

1600

In [50]:
imgs = [int(f.strip()[:-4]) for f in open("imgids.txt","r").readlines()]

In [51]:
len(imgs)

25310

In [52]:
imgs;

In [53]:
predictions = np.argmax(smatx,axis=1)

In [54]:
predictions[:10]

array([ 92, 407,  36,  36, 179, 252,  53, 126,  86, 227])

In [55]:
predictions.shape

(25310,)

In [56]:
smatx[0][92]

0.427672803401947

In [57]:
smatx[2][36]

0.7019358277320862

In [58]:
avocab[36]

'helmet'

In [59]:
namepred = [avocab[nid-1] for nid in predictions]

In [65]:
fullsysdf = pd.DataFrame(zip(imgs,namepred),columns=['vg_img_id','full_pred_name'])

In [84]:
fullsysdf.head()

,vg_img_id,full_pred_name
0,2357077,player
1,2378759,hoodie
2,2319953,giraffe
3,2361401,giraffe
4,2401994,sandwich


In [88]:
mdf = dfsys2.merge(fullsysdf,left_on='image_id',right_on='vg_img_id')

In [122]:
len(mdf)

1071

In [123]:
len(mdf.query('pred_name == full_pred_name'))

1036

In [124]:
len(mdf.query('pred_name != full_pred_name'))

35

In [35]:
df.head()

,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,url,...,responses_domains_r3,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,spellchecked_min2,verified
0,2357077,person,athlete.n.01,batter,"{'man': 4, 'batter': 2, 'baseball player': 1, 'player': 1, 'person': 1}",{},people,people,"{'people': 0.889, '$baseball player': 0.111}",http://object-naming-amore.upf.edu//2357077_815223_singleton_obj.png,...,"{'people': 0.75, '$bat': 0.125, 'clothing': 0.125}","{'player': 3, 'man': 2, 'batter': 1, 'bat': 1, 'helmet': 1}",people,815223,"Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1})","Counter({'man': 11, 'batter': 8, 'player': 5, 'person': 4, 'baseball player': 2, 'helmet': 2})","{'batter': {'cluster': ('baseball player', 'batter', 'man', 'person', 'player'), 'can_be_same_object': ('baseball player', 'batter', 'man', 'person', 'player'), 'adequacy': 1.0..."
1,2378759,article of clothing,overgarment.n.01,jacket,"{'jacket': 3, 'sweatshirt': 2, 'man': 2, 'hoodie': 1, 'person': 1}",{},clothing,clothing,"{'clothing': 0.333, '$sweatshirt': 0.222, 'people': 0.333, '$hoodie': 0.111}",http://object-naming-amore.upf.edu//2378759_1365136_supercat_unique.png,...,"{'clothing': 0.556, 'people': 0.222, '$sweatshirt': 0.111, '$hoodie': 0.111}","{'jacket': 4, 'man': 2, 'sweatshirt': 1, 'hoodie': 1, 'shirt': 1}",clothing,1365136,"Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2, 'shirt': 1})","Counter({'jacket': 14, 'man': 11, 'sweatshirt': 4, 'hoodie': 4, 'person': 2})","{'hoodie': {'cluster': ('hoodie', 'jacket', 'sweatshirt'), 'can_be_same_object': ('hoodie', 'jacket', 'sweatshirt'), 'adequacy': 1.0, 'inadequacy_type': None, 'cluster_id': 0, ..."
2,2319953,animal,ungulate.n.01,giraffe,{'giraffe': 7},{'#bbox': 2},animals_plants,animals_plants,{'animals_plants': 1.0},http://object-naming-amore.upf.edu//2319953_3089834_singleton_obj.png,...,"{'animals_plants': 0.889, 'people': 0.111}","{'giraffe': 8, 'woman': 1}",animals_plants,3089834,"Counter({'giraffe': 32, 'woman': 1})","Counter({'giraffe': 32, 'woman': 1})","Counter({'giraffe': 32, 'woman': 1})","Counter({'giraffe': 32, 'woman': 1})",Counter({'giraffe': 32}),"{'giraffe': {'cluster': ['giraffe'], 'adequacy': 1.0, 'can_be_same_object': ['giraffe'], 'inadequacy_type': None, 'cluster_id': 0, 'cluster_weight': 1.0}}"
3,2361401,animal,ungulate.n.01,giraffe,{'giraffe': 9},{},animals_plants,animals_plants,{'animals_plants': 1.0},http://object-naming-amore.upf.edu//2361401_2363103_singleton_obj.png,...,{'animals_plants': 1.0},{'giraffe': 9},animals_plants,2363103,Counter({'giraffe': 36}),Counter({'giraffe': 36}),Counter({'giraffe': 36}),Counter({'giraffe': 36}),Counter({'giraffe': 36}),"{'giraffe': {'cluster': ['giraffe'], 'adequacy': 1.0, 'can_be_same_object': ['giraffe'], 'inadequacy_type': None, 'cluster_id': 0, 'cluster_weight': 1.0}}"
4,2401994,"food, nutrient",dish.n.02,sandwich,"{'sandwich': 6, 'food': 2, 'hotdog': 1}",{},food,food,"{'food': 0.889, '$hotdog': 0.111}",http://object-naming-amore.upf.edu//2401994_1141828_singleton_obj.png,...,"{'food': 0.778, '$foot': 0.111, '$burger': 0.111}","{'food': 3, 'sandwich': 3, 'foot': 1, 'cake': 1, '

In [108]:
hum_error_cats = Counter()
hum_error_cats2 = Counter()

sys_error_cats = Counter()
sys_error_cats2 = Counter()


for ix,row in mdf.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh['canon']
    topname = resp.most_common(1)[0][0]
    verif = rowh['verified']
    
    #print(topname)
    for name in resp:
        err = get_error(topname,name,verif,resp)
        if name in vocab:
            hum_error_cats[err] += resp[name]

        if name in avocab:
            hum_error_cats2[err] += resp[name]
    
        
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys_error_cats[sys_err] += 1
    
        
    sys_err = get_error(topname,row['full_pred_name'],verif,resp)
    sys_error_cats2[sys_err] += 1

In [118]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
        ["humans, 442 vocab"]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
        ["FRCNN, 442 vocab"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats],

        ["humans, 1600 vocab"]+[hum_error_cats2[t]/sum(hum_error_cats2.values()) for t in relevant_cats],
    ["FRCNN, 1600 vocab"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats]

]

In [119]:
rdf = pd.DataFrame(table,columns=['model']+relevant_cats)

In [120]:
rdf

,model,hit,adequate,low_count,bounding box,off???
0,"humans, 442 vocab",0.758558,0.146164,0.042169,0.043843,0.000000
1,"FRCNN, 442 vocab",0.749767,0.122316,0.023343,0.018674,0.077498
2,"humans, 1600 vocab",0.736372,0.153985,0.052731,0.046715,0.000000
3,"FRCNN, 1600 vocab",0.738562,0.117647,0.024276,0.021475,0.088702


In [98]:
avocab

['yolk',
 'goal',
 'bathroom',
 'macaroni',
 'umpire',
 'toothpick',
 'alarm clock',
 'ceiling fan',
 'photos',
 'parrot',
 'tail fin',
 'birthday cake',
 'calculator',
 'catcher',
 'toilet',
 'batter',
 'stop sign,stopsign',
 'cone',
 'microwave,microwave oven',
 'skateboard ramp',
 'tea',
 'dugout',
 'products',
 'halter',
 'kettle',
 'kitchen',
 'refrigerator,fridge',
 'ostrich',
 'bathtub',
 'blinds',
 'court',
 'urinal',
 'knee pads',
 'bed',
 'flamingo',
 'giraffe',
 'helmet',
 'giraffes',
 'tennis court',
 'motorcycle',
 'laptop',
 'tea pot',
 'horse',
 'television,tv',
 'shorts',
 'manhole',
 'dishwasher',
 'jeans',
 'sail',
 'monitor',
 'man',
 'shirt',
 'car',
 'cat',
 'garage door',
 'bus',
 'radiator',
 'tights',
 'sailboat,sail boat',
 'racket,racquet',
 'plate',
 'rock wall',
 'beach',
 'trolley',
 'ocean',
 'headboard,head board',
 'tea kettle',
 'wetsuit',
 'tennis racket,tennis racquet',
 'sink',
 'train',
 'keyboard',
 'sky',
 'match',
 'train station',
 'stereo',
 'b

In [ ]:
df.iloc½